In [3]:
import shutil
import os
import re

import tensorflow as tf
import os
import sys
import random
import numpy as np

from tqdm import tqdm #progress bar
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

PATH_BENIGN = 'C:/Users/Marta/Desktop/MI Medical Image Segmentation Project/Breast Cancer Segmentation Project/data/data/benign/'
PATH_MALIGNANT = 'C:/Users/Marta/Desktop/MI Medical Image Segmentation Project/Breast Cancer Segmentation Project/data/data/malignant/'
PATH_NORMAL = 'C:/Users/Marta/Desktop/MI Medical Image Segmentation Project/Breast Cancer Segmentation Project/data/data/normal/'

def check_if_folder_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)


def przenies_pliki(folder_zrodlowy):
    check_if_folder_exists(folder_zrodlowy+'images')
    check_if_folder_exists(folder_zrodlowy+'masks')

    for plik in os.listdir(folder_zrodlowy): #listuje pliki w katalogu zrodlowym
        if (not os.path.isdir(folder_zrodlowy + plik)):
            if re.match(r'.*mask.*', plik): #regex, dopasowane wszystkie stringi ktore w srodku maja napis mask, a na lewo i prawo od napisu mask maja od 0 do nieskonczonosci dowolnych znakow
                shutil.move(folder_zrodlowy + plik, folder_zrodlowy + 'masks/' + plik) #przenosimy plik do folderu z maskami
                #print(plik)
            else:
                shutil.move(folder_zrodlowy + plik, folder_zrodlowy + 'images/' + plik)
                

przenies_pliki(PATH_BENIGN) #wyswitlenie nazw plikow ktore maja mask w nazwie
przenies_pliki(PATH_MALIGNANT)
przenies_pliki(PATH_NORMAL)


In [4]:
PATH_TRAINING = 'C:/Users/Marta/Desktop/MI Medical Image Segmentation Project/Breast Cancer Segmentation Project/data/data/training/'
PATH_TESTING = 'C:/Users/Marta/Desktop/MI Medical Image Segmentation Project/Breast Cancer Segmentation Project/data/data/testing/'

def create_directory_structure(test_dir, train_dir):
    check_if_folder_exists(test_dir + 'benign/' + 'images')
    check_if_folder_exists(test_dir + 'malignant/' + 'images')
    check_if_folder_exists(test_dir + 'normal/' + 'images')

    check_if_folder_exists(test_dir + 'benign/' + 'masks')
    check_if_folder_exists(test_dir + 'malignant/' + 'masks')
    check_if_folder_exists(test_dir + 'normal/' + 'masks')

    check_if_folder_exists(train_dir + 'benign/' + 'images')
    check_if_folder_exists(train_dir + 'malignant/' + 'images')
    check_if_folder_exists(train_dir + 'normal/' + 'images')

    check_if_folder_exists(train_dir + 'benign/' + 'masks')
    check_if_folder_exists(train_dir + 'malignant/' + 'masks')
    check_if_folder_exists(train_dir + 'normal/' + 'masks')


create_directory_structure(PATH_TESTING, PATH_TRAINING)
    


In [5]:
def split_data(test_dir, train_dir, source_dir, split_ratio = 0.2):
    images_names = os.listdir(source_dir + 'images') #lista z nazwami imagesów
    images_count = len(images_names) 
    test_count = int(split_ratio * images_count)
    for i in range(0, test_count):
        random_number = random.randint(0, images_count - i) #z kazda iteracja zmniejsza sie liczba dostepnych obrazow
        images_count -= 1
        moved_image = images_names.pop(random_number)
        #print(moved_image) #nazwy tych co przeniesiemy do zbioru testujacego
        masks_to_move = [] #pusta lista
        masks_names = os.listdir(source_dir + 'masks') #mam nazwy masek: z benignow biore do listy nazwy zdjec z maskami
        pattern = moved_image.split(".")[0] #to co przed ".png"
        pattern = re.escape(pattern)        

        for mask in masks_names:
            if re.match(fr'{pattern}.*', mask): #regex, dopasowane wszystkie stringi ktore w srodku maja napis mask, a na lewo i prawo od napisu mask maja od 0 do nieskonczonosci dowolnych znakow
                shutil.move(source_dir + 'masks/' + mask, test_dir + 'masks/' + mask) #przenosimy plik do folderu z maskami
                #print(mask + "and" + pattern )

        shutil.move(source_dir + 'images/' + moved_image, test_dir + 'images/' + moved_image)

    images_names = os.listdir(source_dir + 'images')
    for image in images_names:
        shutil.move(source_dir + 'images/' + image, train_dir + 'images/' + image)


    masks_names = os.listdir(source_dir + 'masks')
    for mask in masks_names:
        shutil.move(source_dir + 'masks/' + mask, train_dir + 'masks/' + mask)
            
            


split_data(PATH_TESTING + 'benign/', PATH_TRAINING + 'benign/', PATH_BENIGN) #30% benigna
split_data(PATH_TESTING + 'malignant/', PATH_TRAINING + 'malignant/', PATH_MALIGNANT)
split_data(PATH_TESTING + 'normal/', PATH_TRAINING + 'normal/', PATH_NORMAL)
    


In [6]:
seed = 37
np.random.seed = seed

img_width = 128
img_height = 128
img_channels = 3

In [49]:
def resize_images(path_to_data):
    subfolder_names = ["normal", "benign", "malignant"]
    
    normal_imgs_file_names = os.listdir(path_to_data + "normal/images")
    benign_imgs_file_names = os.listdir(path_to_data + "benign/images")
    malignant_imgs_file_names = os.listdir(path_to_data + "malignant/images")

    normal_masks_file_names = os.listdir(path_to_data + "normal/masks")
    benign_masks_file_names = os.listdir(path_to_data + "benign/masks")
    malignant_masks_file_names = os.listdir(path_to_data + "malignant/masks")

    all_img_file_names = [normal_imgs_file_names, benign_imgs_file_names, malignant_imgs_file_names]
    all_mask_file_names = [normal_masks_file_names, benign_masks_file_names, malignant_masks_file_names]

    number_of_training_imgs = len(normal_imgs_file_names) + len(benign_imgs_file_names) + len(malignant_imgs_file_names)

    #i fill matrix with zeros firstly
    #independent, images from trainng set
    X_matrix = np.zeros((number_of_training_imgs, img_height, img_width, img_channels), dtype=np.uint8)

    #dependent, what i try to predict. 1 is bool (yes or no). Masks from training set
    #idk but i guess that 3 will be for 3 classes normal, benign and malignant
    Y_matrix = np.zeros((number_of_training_imgs, img_height, img_width, 3), dtype=bool)
    print(Y_matrix.shape)
    print('Resizing training images and masks')

    counter = 0
    for class_id, subfolder in enumerate(subfolder_names):
        #print(class_id, subfolder)
        current_imgs_file_names = all_img_file_names[class_id] #wszystkie nazwy zdjec np z benign
        current_masks_file_names = all_mask_file_names[class_id]
        # if(class_id == 0):
        #     continue
        for img_file_name in tqdm(current_imgs_file_names): 
            # if not(img_file_name == "benign (4).png"):
            #     continue
            path = path_to_data + f"{subfolder}/images/" + img_file_name #iterujemy po plikach np w benign
            img = imread(path)[:,:,:img_channels] 
            img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
            X_matrix[counter] = img #all images
            
            
            mask = np.zeros((img_height, img_width, 3), dtype=bool) #tensor o takich wymiarach, wszystkie komorki ustwiam na zero
            pattern = img_file_name.split(".")[0] #np benign (1)
            pattern = re.escape(pattern) #nie traktowal nawiasow jako special character
            corresponding_masks = [curr_mask for curr_mask in current_masks_file_names if re.match(fr'{pattern}.*', curr_mask)] #dodaj curr_mask do listy jesli spelnia warunki    
            # print(img_file_name)
            for mask_file in corresponding_masks:
                mask_ = imread(path_to_data + f'{subfolder}/masks/' + mask_file, as_gray=True) #wczytanie zdjecia w skali szarosci

                mask_ = np.expand_dims(resize(mask_, (img_height, img_width), mode='constant',  #resize do 128x128 i dodanie 3 kanału
                                             preserve_range=True), axis=-1)
                
                mask_ = np.repeat(mask_, repeats=3, axis=-1)    #zrobienie 128x128x3
                mask_ = mask_.astype("bool")                    #
                #powyzej kazdy z 3 kanalow jest taki sam jak maska
                # #mask_[:,:,1] = 0.5
                # print(mask_file, np.unique(mask_[:,:,1], return_counts=True))
                mask = np.maximum(mask, mask_)  #merging masks to get 1 mask. At every pixel i look for max value, take this value and create a mask
            # print("\n\n\n",np.unique(mask[:,:,1], return_counts=True))
            mask_ = mask_.astype("float")
            if(class_id == 0):
                mask[:,:,0] = 1 - mask[:,:,0]
            elif(class_id == 1):
                mask[:,:,2] = 0
                mask[:,:,0] = 1 - mask[:,:,0]
            else:
                mask[:,:,1] = 0
                mask[:,:,0] = 1 - mask[:,:,0]
            # print("normal",mask_file,np.unique(mask[:,:,0], return_counts=True))
            # print("brajan",mask_file,np.unique(mask[:,:,1], return_counts=True))
            # print("marian",mask_file,np.unique(mask[:,:,2], return_counts=True))
            # break
            Y_matrix[counter] = mask
            counter+=1
    return X_matrix, Y_matrix
# train_ids = next(os.walk(train_path))[1] #name of folder with train data
# test_ids = next(os.walk(test_path))[1]
#start from [1] bc otherwise it would be name stage1_train



X_train, Y_train = resize_images(PATH_TRAINING)
X_test, Y_test = resize_images(PATH_TESTING)

(625, 128, 128, 3)
Resizing training images and masks


  0%|          | 0/107 [00:00<?, ?it/s]

100%|██████████| 168/168 [00:06<00:00, 25.20it/s]


(155, 128, 128, 3)
Resizing training images and masks


100%|██████████| 42/42 [00:02<00:00, 15.20it/s]


In [51]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((625, 128, 128, 3),
 (625, 128, 128, 3),
 (155, 128, 128, 3),
 (155, 128, 128, 3))

In [ ]:
print('Resizing training images and masks')
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):   
    path = train_path + id_ #id_ is path to image
    img = imread(path + '/images/' + id_ + '.png')[:,:,:img_channels]  
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    X_train[n] = img  #Fill empty X_train with values from img
    mask = np.zeros((img_height, img_width, 1), dtype=bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (img_height, img_width), mode='constant',  
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)  #merging masks to get 1 mask. At every pixel i look for max value, take this value and create a mask
            
    Y_train[n] = mask   

# test images
X_test = np.zeros((len(test_ids), img_height, img_width, img_channels), dtype=np.uint8)
sizes_test = []
print('Resizing test images') 
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = test_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:img_channels]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (img_height, img_width), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

#randomly pick image and show it with associated mask
image_x = random.randint(0, len(train_ids))
imshow(X_train[image_x])
plt.show()
imshow(np.squeeze(Y_train[image_x]))
plt.show()



In [ ]:
#building U-net model

#encoder path
#first layer
inputs = tf.keras.layers.Input((img_width, img_height, img_channels)) #0-255

#convert pixel into floating point number bc keras layers deal with floating numbers
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#convolutional layers
#padding=same to have the same output image size as input
c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(s) 
#kernel_initializer to start with some weights,
#he_normal does it by truncated normal distribution (centered around zero)


#drop 10% pixels
c1 = tf.keras.layers.Dropout(0.1)(c1)

#second convolutional layer
c1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1) #kernel_initializer to start with some weights, he_normal does it by truncated normal distribution (centered around zero)


#max pooling
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

###
c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.1)(c3)
c3 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.1)(c4)
c4 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.1)(c5)
c5 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
p5 = tf.keras.layers.MaxPooling2D((2,2))(c5)

#decoder path

#Conv2DTranspose is opposite to Conv2D
u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.2)(c8)
c8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1])
c9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.2)(c9)
c9 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(c9)

#optimizer tries to minimize loss function, accuracy to measure model's performance after training
#adam works like backpropagation to train model, can also be stochastic gradient descent
#loss=binary_crossentropy bc pixel belongs to mask or not
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

#callbacks
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.keras', verbose=1, save_best_only=True)
#monituruje postep modelu i zapisuje do pliku. verbose = 1 oznacza, ze podczas zapisywania modelu bedzie wyswietlany komunikat
#save_best_only aby zapisac najlepszą wersję modelu


#trening zatrzymany po 2 kolejnych epokach, w których wartość val_loss (strat walidacyjnych) nie ulegnie poprawie. 
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs') #to create logs
]

#training model
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=25, callbacks=callbacks)



In [ ]:
#####

idx = random.randint(0, len(X_train))

# - predykcja na zbiorze treningowym `X_train` za pomocą wytrenowanego modelu.
# - Przy użyciu metody `predict` model oblicza prognozy na podstawie danych treningowych.
# - Tutaj wykorzystywane jest pierwsze 90% danych treningowych. X_train.shape[0] to liczba przykladow. Biorę od pierwszego przykladu "":"" do 90% wszystkich
#Dla verbose=1: W trakcie predykcji wyświetlane będą informacje postępu, takie jak numer aktualnie przetwarzanego przykładu. Dzięki temu można śledzić postęp operacji i zobaczyć, ile czasu pozostało do ukończenia procesu predykcji.
#Dla verbose=2: Oprócz informacji wyświetlanych dla verbose=1, wyświetlane będą dodatkowe szczegóły, takie jak czas trwania poszczególnych kroków i inne informacje diagnostyczne.
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)

#ostatnie 10% danych bedzie walidacyjne
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)

#testing model on data that were not used during model training
preds_test = model.predict(X_test, verbose=1)

#Predykcje przekraczające wartość 0.5 są traktowane jako piksele o wartości 1, a pozostałe 
#jako piksele o wartości 0 (przy użyciu astype(np.uint8)).
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)


# check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix]) #obraz treningowy dla losowego indeksu ix
plt.show()
imshow(np.squeeze(Y_train[ix])) #rzeczywista maska dla obrazu treningowego o indeksie ix
plt.show()
imshow(np.squeeze(preds_train_t[ix])) #predykcja modelu dla obrazu treningowego o indeksie ix
plt.show()

#check on some random validation samples
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()